In [2]:
import ee
import geemap
import hvplot.pandas
import hvplot.xarray
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np

In [3]:
ee.Authenticate()

True

In [29]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [148]:
kwargs = {
}
# Set timespan
start_date, end_date = '2000-01-01', '2024-01-30'

# Cloud masking (scene-based)
cld_filt_thresh = 80        # Maximum image cloud cover percent allowed in image collection

# water masking
mask_water = False

# # Cloud masking (pixel-based, s2cloudless only)
# cld_prb_thresh = 25      # Cloud probability (%); values greater than are considered cloud
# cld_prj_dist = 1          # Maximum distance (km) to search for cloud shadows from cloud edges (based on Hollstein decision tree)

# Cloud masking (pixel-based, cloud score+ only)
qa_band = 'cs_cdf'
clear_thresh = 0.75

In [105]:
geemap.ee_to_gdf(stations).to_csv('stations.csv')

In [126]:
# load 3S basin
json_data = 'geometries/geoms.geojson'
fc_geoms = geemap.geojson_to_ee(json_data)
roi_geom = fc_geoms.first().geometry()

# load dams
df = pd.read_csv('geometries/3SReservoirs.csv')
gdf_dams = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_xy(df['X'], df['Y']), crs=4326).drop(columns=['X', 'Y']).set_index('id')
fc_dams = geemap.geopandas_to_ee(gdf_dams)

# define stations
stations = ee.FeatureCollection([
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([106.39220569493087, 14.11961739545873]), {'station_id': '430102', 'station_name': 'Siempang'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.78237137912318, 12.897991717109619]), {'station_id': '451305', 'station_name': 'Ban Don'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([106.5278998223061, 13.553039642985812]), {'station_id': '450101', 'station_name': 'Lum Phat'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.47041987706022, 13.940093424442852]), {'station_id': '440202', 'station_name': 'Pleiku'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.10639453530615, 14.050489408398247]), {'station_id': '440103', 'station_name': 'Andaung Meas'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.44760623609672, 13.792268298200513]), {'station_id': '440100', 'station_name': 'Phum Pi'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([105.987503299606, 13.537303972542501]), {'station_id': '014501', 'station_name': 'Stung Treng'}),
])

lake_rhone = ee.Feature(ee.FeatureCollection('users/michaelbrechbuehler/Landsat_ST/shapefiles/rhone_wgs84').first().geometry().centroid(), {'station_id': 'Rhonesee'})
lake_steisee = ee.Feature(ee.FeatureCollection('users/michaelbrechbuehler/Landsat_ST/shapefiles/steisee_wgs84').first().geometry().centroid(), {'station_id': 'Steinsee'})

# add S2 clear median
filters = ee.Filter.And(
    ee.Filter.bounds(roi_geom), 
    ee.Filter.date('2017-12-01', '2019-03-31'),
    ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)
)
s2_median = ee.ImageCollection("COPERNICUS/S2_SR").filter(filters).median()

Map.addLayer(s2_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'Sentinel-2 RGB')
Map.addLayer(roi_geom, {'color': 'yellow'}, '3S Basin')
Map.addLayer(stations, {'color': 'red'}, 'Stations')
Map.add_gdf(gdf_dams, 'Dams', {'color': 'blue'})
Map.centerObject(stations)

## Get point samples at virtual stations

In [110]:
stations = stations.filter(ee.Filter.eq('station_name', 'Lum Phat'))

In [156]:
%load_ext autoreload
%autoreload 2
import functions_process as funcs_process
import functions_turbidity as funcs_turb
import functions_sampling as funcs_sampling

for idx, row in geemap.ee_to_df(stations).iterrows():
    name, id = row.station_name, row.station_id
    print(f'{name} ({id})')
    station = stations.filter(ee.Filter.eq('station_id', id))

    # get Rrs imagecolls
    ic_msi = funcs_process.load_rrs_imcoll(sensor='msi', start_date=start_date, end_date=end_date, mask_water=mask_water, bounds=station)
    ic_oli = funcs_process.load_rrs_imcoll(sensor='oli', start_date=start_date, end_date=end_date, mask_water=mask_water, bounds=station)
    ic_etm = funcs_process.load_rrs_imcoll(sensor='etm', start_date=start_date, end_date=end_date, mask_water=mask_water, bounds=station)
    ic_all = ic_msi.merge(ic_oli).merge(ic_etm)
    
    # comput tsm features
    ic_all = ic_all \
        .map(funcs_turb.calc_spm_nechad) \
        .map(funcs_turb.calc_tur_nechad) \
        .map(funcs_turb.calc_tur_dogliotti) \
        .map(funcs_turb.calc_indices)

    # merge imagecolls and sample
    fc_all = ee.FeatureCollection(ic_all.map(funcs_sampling.sample_image(station))).flatten()

    # export to drive
    geemap.ee_export_vector_to_drive(
        fc_all.filter(ee.Filter.gt('roi_coverage', 90)),
        fileFormat='CSV', 
        folder="export",
        description=f"rrs_samples_{name.replace(' ', '').lower()}_{id}", 
    )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Siempang (430102)
Exporting rrs_samples_siempang_430102... Please check the Task Manager from the JavaScript Code Editor.
Ban Don (451305)
Exporting rrs_samples_bandon_451305... Please check the Task Manager from the JavaScript Code Editor.
Lum Phat (450101)
Exporting rrs_samples_lumphat_450101... Please check the Task Manager from the JavaScript Code Editor.
Pleiku (440202)
Exporting rrs_samples_pleiku_440202... Please check the Task Manager from the JavaScript Code Editor.
Andaung Meas (440103)
Exporting rrs_samples_andaungmeas_440103... Please check the Task Manager from the JavaScript Code Editor.
Phum Pi (440100)
Exporting rrs_samples_phumpi_440100... Please check the Task Manager from the JavaScript Code Editor.
Stung Treng (014501)
Exporting rrs_samples_stungtreng_014501... Please check the Task Manager from the JavaScript Code Editor.


In [122]:
# fetch server-side data and convert to dataframe
data = fc_all.filter(ee.Filter.gt('roi_coverage', 90)).getInfo()
df_all = pd.DataFrame.from_dict(data['features'])
df_all = df_all.join(df_all["properties"].apply(pd.Series)).drop(columns=['geometry', 'properties'])
df_all['timestamp'] = pd.to_datetime(df_all['system:time_start'], unit='ms')

In [ ]:
# plot with hvplot
df_all['red_median'] = np.where(df_all['platform']=='LANDSAT-7', df_all['B3_median'], df_all['B4_median'])
df_all['nir_median'] = np.where(df_all['platform']=='LANDSAT-7', df_all['B4_median'], df_all['B5_median'])
df_all.hvplot.line(x='timestamp', y='red_median', groupby='station_id') *\
df_all.hvplot.scatter(x='timestamp', y='red_median', color='platform', groupby='station_id')

In [ ]:
# plot with hvplot
df_all.hvplot.line(x='timestamp', y='add_spm_B4_nechad_median', groupby='station_id') *\
df_all.hvplot.scatter(x='timestamp', y='add_spm_B4_nechad_median', color='platform', groupby='station_id')

## Get map over virtual station

In [ ]:
bounds = ee.Algorithms.GeometryConstructors.Point(gdf_dams.iloc[0].geometry.coords[0]).buffer(5000).bounds()

In [ ]:
%load_ext autoreload
%autoreload 2
import functions_process as funcs_process
import functions_turbidity as funcs_turb
import functions_sampling as funcs_sampling

# get Rrs imagecolls
ic_msi = funcs_process.load_rrs_imcoll(sensor='msi', start_date=start_date, end_date=end_date, bounds=bounds)
ic_oli = funcs_process.load_rrs_imcoll(sensor='oli', start_date=start_date, end_date=end_date, bounds=bounds)
ic_etm = funcs_process.load_rrs_imcoll(sensor='etm', start_date=start_date, end_date=end_date, bounds=bounds)
ic_all = ic_msi.merge(ic_oli).merge(ic_etm)

# comput tsm features
ic_all = ic_all \
    .map(funcs_turb.calc_spm_nechad) \
    .map(funcs_turb.calc_tur_nechad) \
    .map(funcs_turb.calc_tur_dogliotti) \
    .map(funcs_turb.calc_indices)

In [ ]:
crs = ic_msi.first().select(0).projection().crs()
scale = 30

# wxee convert to xarray
#ds_msi = ic_msi.select('B4').limit(25).wx.to_xarray(scale=scale, region=bounds)

# geemap export
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(ic_msi).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)

In [ ]:
# Export red bands to geotiffs
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_etm).select('B3'), folder='export/msi', maxPixels=200000000, region=bounds, scale=30)
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_oli).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_msi).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)

In [ ]:
ee.Algorithms.GeometryConstructors.Point(gdf_dams.iloc[0].geometry.coords[0]).buffer(5000).bounds()

In [ ]:
ds_msi.is_cloud.isel(time=0).plot()

In [ ]:
Map.centerObject(img_oli, 14)

In [ ]:
img_etm = ic_etm.sort('CLOUD_COVER', True).first()
img_oli = ic_oli.sort('CLOUD_COVER', True).first()
img_msi = ic_msi.sort('CLOUDY_PIXEL_PERCENTAGE', True).first()

#vis = {'bands': 'Rrs_B4', 'min': 0, 'max': 0.2}
Map.addLayer(ee.Image(img_etm),  {}, 'ETM+, Rrs_B3')
Map.addLayer(ee.Image(img_oli),  {}, 'OLI, Rrs_B4')
Map.addLayer(ee.Image(img_msi),  {}, 'MSI, Rrs_B4')
Map.centerObject(img_msi, 14)